In [85]:
import numpy as np
import pint

u = pint.UnitRegistry(autoconvert_offset_to_baseunit=True)

class ombede:
    
    GEAR_RADII = {
        0: 0.0  * u.cm,
        1: 10.0 * u.cm,
        2: 18.0 * u.cm,
        3: 28.5 * u.cm,
        4: 40.0 * u.cm,
        5: 50.0 * u.cm,
    }
    
    ENGINE_DISPLACEMENT = 1.0 * u.l
    VOLUMETRIC_EFFICIENCY = 0.75

    default_params = {
        "COOLANT_TEMP":             96.0 * u.degC,
        "INTAKE_PRESSURE":          23.0 * u.kPa,
        "RPM":                      2222.0 / u.min,
        "SPEED":                    38.0 * u.km/u.hour,
        "TIMING_ADVANCE":           16.0 * u.deg,
        "INTAKE_TEMP":              40.0 * u.degC,
        "FUEL_LEVEL":               100.0,
        "DISTANCE_SINCE_DTC_CLEAR": 3676.0 * u.km,
        "CATALYST_TEMP_B1S1":       466.29998779296875 * u.celsius,
        "ABSOLUTE_LOAD":            13.72549057006836,
        "COMMANDED_EQUIV_RATIO":    89.9993896484375,
        "THROTTLE_ACTUATOR":        9.803921699523926,
        'ETHANOL_PERCENT':          25.0,
    }
    
    dependencies = {
        'GEAR_RADIUS': ['SPEED', 'RPM'],
        'GEAR': ['GEAR_RADIUS'],
        'MAF_A': ['ABSOLUTE_LOAD', 'RPM'],
        'MAF_B': ['INTAKE_PRESSURE', 'INTAKE_TEMP', 'RPM'],
        'AFR': ['COMMANDED_EQUIV_RATIO', 'ETHANOL_PERCENT'],
        'FUEL_FLOW': ['AFR', 'MAF_A'],
        'FUEL_EFFICIENCY': ['SPEED', 'FUEL_FLOW'],
    }
    
    data = {}
    
    def _solve_dependencies(self, quant, level=np.inf):
        """ Replace quantity in list by its dependency tree """
        if quant in self.dependencies.keys() and level > 0:
            return [[self._solve_dependencies(d,level=level-2) for d in self.dependencies[dep]] if (dep in self.dependencies.keys() and level > 1) else dep for dep in self.dependencies[quant]]
        else:
            return quant
        
    def _flatten(self, x):
        """ Flatten irregular list of lists """
        if type(x) == list:
            return [a for i in x for a in self._flatten(i)]
        else:
            return [x]
    
    def get_dependencies(self, quantities, level=np.inf):
        """ Return flat list of unique dependencies.
            The deepest level np.inf corresponds to quantities to be read from the OBD interface. """
        return list(set(self._flatten([self._solve_dependencies(q, level=level) for q in quantities])))
    
    def request(self, quantities):
        # Get data from OBD interface
        self.data = {k:self.default_params[k] for k in self.get_dependencies(quantities)}
        
        # Compute quantities from lower to higher in the dependency tree levels
        for level in np.arange(3,-1,-1):
            self.compute_quantities(self.get_dependencies(quantities, level=level))
            
        return {k:self.data[k] for k in quantities}
    
    def compute_quantities(self, quantities):
        
        if 'GEAR_RADIUS' in quantities:
            self.compute_gear_radius()
            
        if 'GEAR' in quantities:
            self.compute_gear()
            
        if 'MAF_A' in quantities:
            self.compute_maf_a()
            
        if 'MAF_B' in quantities:
            self.compute_maf_b()
            
        if 'AFR' in quantities:
            self.compute_afr()
            
        if 'FUEL_FLOW' in quantities:
            self.compute_fuel_flow()
            
        if 'FUEL_EFFICIENCY' in quantities:
            self.compute_fuel_efficiency()
            
    def compute_gear_radius(self):
        self.data['GEAR_RADIUS'] = (self.data['SPEED']/self.data['RPM']).to(u.cm)
        
    def compute_gear(self):
        diff = 100.*u.cm
        self.data['GEAR'] = None
        for k in self.GEAR_RADII.keys():
            new_diff = np.abs(self.GEAR_RADII[k] - self.data['GEAR_RADIUS'])
            if  new_diff < diff:
                diff = new_diff
                self.data['GEAR'] = k
                
    def compute_maf_a(self):
        # mass_air_flow [g/s] = 1.184 [g/l] * displacement [l/intake stroke] * load_abs / 100 * engine_speed [r/min] / 2 [r/intake stroke] / 60 [sec/min]
        self.data['MAF_A'] = ((1.184 * u.g/u.l) * self.ENGINE_DISPLACEMENT * self.data['ABSOLUTE_LOAD']/100 * self.data['RPM'] / 2).to('g/s')

    def compute_maf_b(self):
        # MAF [g/s] = (intake pressure/intake temp) * (molecular mass of air/gas constant for air) * (RPM/60) * (engine displacement/2) * volumetric efficiency
        self.data['MAF_B'] = (self.data['INTAKE_PRESSURE'] / self.data['INTAKE_TEMP'].to('kelvin') * ((28.97 * u.g/u.mol)/(8.3144598 * u.joules/u.K/u.mol)) * self.data['RPM'] * self.ENGINE_DISPLACEMENT/2 * self.VOLUMETRIC_EFFICIENCY).to('g/s')

    def compute_afr(self):
        # for now, assuming commanded EQ = actual
        self.data['AFR'] = u.g/u.ml * (14.7  + (9.0  - 14.7) * self.data['ETHANOL_PERCENT']/100) * self.data['COMMANDED_EQUIV_RATIO']/100

    def compute_fuel_flow(self):
        self.data['FUEL_FLOW'] = (self.data['MAF_A'] / self.data['AFR'] / 0.800).to('l/hour')

    def compute_fuel_efficiency(self):
        self.data['FUEL_EFFICIENCY'] = (self.data['SPEED'] / self.data['FUEL_FLOW']).to('km/l')

In [86]:
ombd = ombede()
ombd.request(['MAF_A', 'MAF_B', 'GEAR_RADIUS', 'FUEL_FLOW', 'FUEL_EFFICIENCY'])

{'MAF_A': <Quantity(3.0091399512736006, 'gram / second')>,
 'MAF_B': <Quantity(3.5539715694111655, 'gram / second')>,
 'GEAR_RADIUS': <Quantity(28.502850285028504, 'centimeter')>,
 'FUEL_FLOW': <Quantity(1.1333937319956089, 'liter / hour')>,
 'FUEL_EFFICIENCY': <Quantity(33.527624979089985, 'kilometer / liter')>}

In [123]:
import obd
obd.logger.setLevel(obd.logging.DEBUG)

In [128]:
connection = obd.OBD('/dev/rfcomm0') # auto-connects to USB or RF port

[obd.obd] ======================= python-OBD (v0.7.1) =======================
[obd.obd] Explicit port defined
[obd.elm327] Initializing ELM327: PORT=/dev/rfcomm0 BAUD=auto PROTOCOL=auto
[obd.elm327] [Errno 13] could not open port /dev/rfcomm0: [Errno 13] Permission denied: '/dev/rfcomm0'
[obd.obd] Closing connection
[obd.obd] Cannot load commands: No connection to car
[obd.obd] ===================================================================


In [119]:
import obd

connection = obd.OBD() # auto-connects to USB or RF port

cmd = obd.commands.SPEED # select an OBD command (sensor)

response = connection.query(cmd) # send the command, and parse the response

print(response.value) # returns unit-bearing values thanks to Pint
print(response.value.to("mph")) # user-friendly unit conversions

[obd.obd] No OBD-II adapters found
[obd.obd] Cannot load commands: No connection to car
[obd.obd] Query failed, no connection available


None


AttributeError: 'NoneType' object has no attribute 'to'